<a href="https://colab.research.google.com/github/coldbilberry/repo-gui/blob/main/%D0%BC%D0%BE%D0%BB%D0%B5%D0%BA%D1%83%D0%BB_%D0%B1%D0%B8%D0%BE_%D1%83%D1%80%D0%BE%D0%BA_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython

In [ ]:
import pandas as pd
from Bio import Entrez
from Bio import SeqIO
from Bio.Seq import MutableSeq
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Seq import MutableSeq
import time

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
gene = SeqIO.read('/content/drive/MyDrive/BIO/gene.fna', 'fasta')
gene.seq

In [ ]:
# переходим с минуса на плюс-нить
gene = gene.seq.complement()
gene

In [ ]:
# находим старт-кодон "ATG"
start_codon_index = str(gene).find("ATG")
start_codon_index

In [ ]:
gene[start_codon_index:]

In [ ]:
# цепочка нуклеотидов со старт кодона
dna_seq = gene[start_codon_index:]
dna_seq

In [ ]:
mutable_seq = MutableSeq(dna_seq)
mutable_seq

In [ ]:
# заменим какой-нибудь кодон после стартового на стоп-кодон
stop_codon = "UAA"

In [ ]:
stop = Seq('UAA')
stop.transcribe().translate()

In [ ]:
# рассмотрим белок из цепи ДНК dna_seq
protein = dna_seq.transcribe().translate()
protein

In [ ]:
mutable_seq[9:12] = stop_codon

In [ ]:
nonsense_seq = mutable_seq.transcribe().translate()
nonsense_seq

In [ ]:
mutable_seq = MutableSeq(dna_seq)
mutable_seq

In [ ]:
protein

In [ ]:
# заменим аминокислоту на 4-й позиции
mutable_seq[9:12] = 'CAA'
missense_seq = mutable_seq.transcribe().translate()
missense_seq

In [ ]:
mutable_seq = MutableSeq(dna_seq)
mutable_seq

In [ ]:
protein

In [ ]:
mutable_seq[9:12]

In [ ]:
mutable_seq[9:12] = "ACC"

In [ ]:
silent_seq = mutable_seq.transcribe().translate()
silent_seq

In [ ]:
import sys

In [ ]:
Entrez.email = "dbsnp-user@nih.gov"
Entrez.api_key = None

eShandle = Entrez.esearch(db="snp",  # search dbSNP
                          term='A2M[All Fields] AND pathogenic[Clinical_Significance] AND missense variant[Function_Class])',
                          # AND (00000.0000[GLOBAL_MAF] : 00000.0100[GLOBAL_MAF])',
                          usehistory="y", #cache result on server for download in batches
                          retmax=20 # return 20 RSID max
                         )
eSresult = Entrez.read(eShandle)
for k in eSresult:
    print (k, ":", eSresult[k])

In [ ]:
Entrez.email = "dbsnp-user@nih.gov"
Entrez.api_key = None

eShandle = Entrez.esearch(db="snp",  # search dbSNP
                          term='CAD[All Fields] AND pathogenic[Clinical_Significance] AND missense variant[Function_Class])',
                          # AND (00000.0000[GLOBAL_MAF] : 00000.0100[GLOBAL_MAF])',
                          usehistory="y", #cache result on server for download in batches
                          retmax=20 # return 20 RSID max
                         )

In [ ]:
eSresult = Entrez.read(eShandle)

In [ ]:
for k in eSresult:
    print (k, ":", eSresult[k])

In [ ]:
from urllib.error import HTTPError

In [ ]:
rslist = (eSresult['IdList'])

In [ ]:
for rs in rslist:
    print(rs)

In [ ]:
webenv = eSresult["WebEnv"]
query_key = eSresult["QueryKey"]
total_count = int(eSresult["Count"])
query_key = eSresult["QueryKey"]
retmax = 2

In [ ]:
fetch_count = 0
for start in range(0, total_count, retmax):
    end = min(total_count, start+retmax)
    print("Going to download record %i to %i" % (start+1, end))
    attempt = 0
    #fetch_count += 1
    while (attempt < 3):
        attempt += 1
        try:
            fetch_handle = Entrez.efetch(db="snp",
                                         #rettype="uilist", #available types [uilist | xml (use retmode=xml))
                                         retmode="xml",
                                         retstart=start,
                                         retmax=retmax,
                                         webenv=webenv,
                                         query_key=query_key )
        except HTTPError as err:
            if 500 <= err.code <= 599:
                print("Received error from server %s" % err)
                print("Attempt %i of 3" % attempt)
                time.sleep(15)
            else:
                raise
    if (fetch_handle):
        #print(fetch_handle)
        data = fetch_handle.read()
        print(data)
        fetch_handle.close()